## EECS 491 Assignment 3

  Yue Shu  
  Spring 2019  
  Prof. Lewicki  

# Exercise 1. MRFs and Images Denoising

In this problem, you will implement the image de-noising example using a Markov Random Field (MRF).  This material on MRFs is covered in the textbook (Barber) in chapter 4.2.5. The lecture and this problem is based on the presentation in Bishop in chapter 8.3, which is available online.

As discussed in class, energy function for this MRF is

$$ E(\mathbf{x}, \mathbf{y}) = h \sum_i x_i - \beta \sum_{i,j} x_i x_j - \eta \sum_i x_i y_i $$

where the binary variables $x_i$ represent the unknown, noise-free image pixels, which are binary, i.e. black or white.  The variables $y_i$ represent the observed noisy pixels, i.e. the pixel could randomly change from black ($=-1$) to white ($=+1$) or vice-versa.  

The corresponding joint probability distribution over the variables is

$$ p(\mathbf{x},\mathbf{y}) = \frac{1}{Z} \exp \left[ -E(\mathbf{x},\mathbf{y}) \right] $$

## 1.1 Derive the equation that specifies the change in the energy equation when one variable changes state.

Started with the original energy function equation $E$, we have: 

$$ E(\mathbf{x}, \mathbf{y}) = h \sum_i x_i - \beta \sum_{i,j} x_i x_j - \eta \sum_i x_i y_i $$

  Now let $x_k'$ denote the state of $x_k$ after changing, and $E'$ denotes the new energy equation.
  Since each pixel of the image we're discussing here is a binary variable, we may simply assume that the change of one variable is either from $+1$ to $-1$ or vice-versa. By flipping the state of $x_i$, we simply flip by sign and thus obtain $x_i' = -x_i$.

  Therefore, to calculate the difference in the energy equation, we simply substract $E'$ from $E$ as below:

  $$ E' - E =    E(\mathbf{x}', \mathbf{y}) -   E(\mathbf{x}, \mathbf{y}) $$ 
  $$ =  h * (x_0 + ... + x_k' + ... + x_n) - \beta * (x_0x_1 + ... + x_0x_k' + ... + x_0x_n + ... + x_k'x_0 + ... + x_k'x_n + ... $$
  $$ + x_nx_0 + ... + x_nx_k' + ... + x_nx_{n-1}) - \eta * (x_0y_0 + ... + x_k'y_k + ... + x_ny_n) $$
  $$ - (h * (x_0 + ... + x_k + ... + x_n) - \beta * (x_0x_1 + ... + x_0x_k + ... + x_0x_n + ... + x_kx_0 + ... + x_kx_n + ... $$
  $$ + x_nx_0 + ... + x_nx_k + ... + x_nx_{n-1}) - \eta * (x_0y_0 + ... + x_ky_k + ... + x_ny_n)) $$
  $$ = h * (x_0 + ... - x_k + ... + x_n) - \beta * (x_0x_1 + ... - x_0x_k + ... + x_0x_n + ... - x_kx_0 + ... - x_kx_n + ... $$
  $$ + x_nx_0 + ... - x_nx_k + ... + x_nx_{n-1}) - \eta * (x_0y_0 + ... - x_ky_k + ... + x_ny_n) $$
  $$ - (h * (x_0 + ... + x_k + ... + x_n) - \beta * (x_0x_1 + ... + x_0x_k + ... + x_0x_n + ... + x_kx_0 + ... + x_kx_n + ... $$
  $$ + x_nx_0 + ... + x_nx_k + ... + x_nx_{n-1}) - \eta * (x_0y_0 + ... + x_ky_k + ... + x_ny_n)) $$
  $$ E' - E = -2hx_k + 2\beta (2 \sum_{j \in nbr(k)} x_j x_k) + 2 \eta x_k y_k = -2hx_k + 4\beta \sum_{j \in nbr(k)} x_j x_k + 2 \eta x_k y_k
  $$

And the final equation $E' - E$ that specifies the change in the energy equation when one variable $x_k$ changes state is:

$$ E' - E = -2hx_k + 4\beta \sum_{j \in nbr(k)} x_j x_k + 2 \eta x_k y_k $$

## 1.2  Write a program to iteratively (or in random order) update the state variables to minimize the energy (maximize the probability).  Explain your code.  Show that the update algorithm minimizes the energy $E(\mathbf{x}, \mathbf{y})$.

First of all let's define the function to calculate the energy difference. For future convenience, I make the output of the function an array that can also tell me the index `k` of the pixel under changing. 

  `pixels`: an array indicating the pixel of our image  
  `nbr`: an array indicating $x_j$  
  `k`: the index of $x_k$  
  `yk`: the original observation of $x_k$  
  `h`, `beta`, `eta`: the parameters  

In [1]:
def energy_diff (pixels, nbr, k, yk, h, beta, eta):
    output = [0, k]
    sum_xjxk = 0
    i = 0
    ## calculating the sum of all xjxk
    for xj in nbr:
        sum_xjxk += xj * pixels[k]
        i+= 1
    output[0] += - 2 * h * pixels[k] + 4 * beta * sum_xjxk + 2 * eta * pixels[k] * yk
    return output

We also need a helper method to retrieve the neighbours of the pixel we are trying to flip. To eliminate unnecessary edge cases, we may just assume the smallest size of the image is 3x3.

  `width`: the width of the input image  
  `height`: the height of the input image  

In [2]:
def get_nbr (pixels, k, width, height):
    ## the height of k in 2D matrix
    i = k / width
    ## the width of k in 2D matrix
    j = k % width
    ## corners
    if i == 0 and j == 0:
        return [pixels[k + 1], pixels[width]]
    elif i == 0 and j == width - 1:
        return [pixels[k - 1], pixels[k + width]]
    elif i == height - 1 and j == 0:
        return [pixels[k + 1], pixels[k - width]]
    elif i == height - 1 and j == width - 1:
        return [pixels[k - 1], pixels[k - width]]
    ## edges
    elif i == 0:
        return [pixels[k + width], pixels[k + 1], pixels[k - 1]]
    elif i == height - 1:
        return [pixels[k - width], pixels[k + 1], pixels[k - 1]]
    elif j == 0:
        return [pixels[k + 1], pixels[k + width], pixels[k - width]]
    elif j == width - 1:
        return [pixels[k - 1], pixels[k + width], pixels[k - width]]
    # body
    else:
        return [pixels[k + 1], pixels[k - 1], pixels[k + width], pixels[k - width]]

Now we can go ahead and setup our algorithm. 

  I'll implement with a simplified version of simulated annealing algorithm we covered in EECS391, with our energy difference function as the heuristic.   
  So when the energy difference is negative, which means the energy is decreasing, we just accept the result and keep descenting; when the energy difference is positive, we accept the result with a probability less than 1.  

In [3]:
import math
import random

## a helper to determine the probability of acceptance
def accept (delta, T):
    return math.exp( - delta / T)

## simulated annealing algorithm
def anneal(pixels, dim, param_anneal, param_energy):
    
    ## setting up the parameters
    T = param_anneal[0]
    T_min = param_anneal[1]
    alpha = param_anneal[2]
    
    h = param_energy[0]
    beta = param_energy[1]
    eta = param_energy[2]
    
    width = dim[0]
    height = dim[1]
    
    ## the result we want
    output = pixels[:]
    
    ## continue annealing as long as the temperature is higher than the terminate value
    while T > T_min:
        ## randomly pick an index k from the pixels
        k = random.randint(0, len(output) - 1)
        ## retrieve the neighbours of k
        nbr = get_nbr(output, k, width, height)
        diff = energy_diff(output, nbr, k, pixels[k], h, beta, eta)
        ap = accept(diff[0], T)
        if ap > random.random():
            output[k] = 0 - output[k]
        T = T * alpha
    return output

Finally let's have a wrapper method to also compute the finalized energy $E(\mathbf{x}, \mathbf{y})$:

In [4]:
def __MRF_denoise__ (pixels, dim, param_anneal, param_energy):
    ## take the output of simulated annealing
    output = anneal(p, dim, param_anneal, param_energy)
    
    ## setup parameters
    h = param_energy[0]
    beta = param_energy[1]
    eta = param_energy[2]
    
    width = dim[0]
    height = dim[1]
    
    ## the sum variables we need for computing the energy equation
    sum_xi = 0
    sum_xixj = 0
    sum_xiyi = 0
    
    ## compute sum of xi
    for xi in output:
        sum_xi += xi
    
    ## compute sum of xiyi
    i = 0
    for xi in output:
        sum_xiyi += xi * (pixels[i])
        i += 1
    
    ## compute sum of xixj
    i = 0
    for xi in output: 
        for xj in get_nbr(output, i, width, height):
            sum_xixj += xi * xj
        i += 1
    
    return h * sum_xi - beta * sum_xixj - eta * sum_xiyi

  According to simulated annealing algorithm, if given appropriate heuristic, the final energy we get should always end at the global optimal, aka the min energy.  
  However, there does exist some uncertainty in real life practice, where there is no guarantee to always get the most optimal answer. We can first have a really simple example to test our output:

In [5]:
param_anneal = [1, 0.00001, 0.9]
param_energy = [0, 1, 2.1]
## assuming the noise is around 10%, and this is supposed to be a very simplified 3x3 all-white image 
p = [-1,-1,-1,1,-1,-1,-1,-1,-1]
dim = [3, 3]
anneal(p, dim, param_anneal, param_energy)

[-1, -1, -1, -1, -1, -1, -1, -1, -1]

In [6]:
__MRF_denoise__(p, dim, param_anneal, param_energy)

-38.7

  According to the output above, we can see that the final output in this case is just the local minimum.  
  Feel free to try the output, it should return the same results each time. We can practice on some more complexed images in the next part.  